# Ollama Tool Calling

for installing required libarary

``` bash
 !pip install langchain_community langchain_ollama 

```

1. Simple Usage of langchain for calling ollama model

In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(
    model = "hermes3",
    keep_alive = -1,
    temparature = 0.7,
    max_tokens = 2048)
template = """Provide a concise, one-line explanation on 
{topic} from the perspective of a {profession}."""
prompt  = ChatPromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()
stream = chain.stream({"topic":"LLMs","profession":"shipping magnate"})
for c in stream:
    print(c,flush=True,end="")

, adapting to changing market conditions and helping steer companies to success.

### Using ollama Model to get required json type output
#### Note
Using words like "strictly" in the system prompt can significantly alter the model's output.

In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json
json_schema = {
    "title":"Person",
    "description":"Identify information about a person",
    "type":"object",
    "properties":{
        "name":{"title":"Name","description":"the person's name","type":"string"},
        "age":{"title":"Age","description":"the person's age","type":"integer"},
       "favorite_food": {
        "title": "favorite_food",
        "description": """A detailed description of the person's most preferred dish, 
including the type of cuisine, specific ingredients, and any unique preparation methods.""",
        "type": "string"
        }
    },
    "required":['name','age'],
}

llm = ChatOllama(
    model="hermes3",
    temperature=0,
    format="json",
    max_new_tokens =512
)
messages = [
    HumanMessage(
    content="Please tell me about a person strictly using the following Json schema:"
    ),
    HumanMessage(content="{schema}"),
    HumanMessage(
    content="tell me  about a person named Jhon who is 35 years old and loves pizza."
    )
]

prompt = ChatPromptTemplate.from_messages(messages=messages)
dumps = json.dumps(json_schema,indent=3)
chain = prompt | llm | JsonOutputParser()
result = chain.invoke({"schema":dumps})
print(result)

{'name': 'Jhon', 'age': 35, 'favoriteFood': 'pizza'}


In [1]:
from langchain_ollama.chat_models import ChatOllama
model = ChatOllama(model="phi3.5", keep_alive=1, format='json')
r =model.stream("hello")
for c in r:
    print(c)

content='{"' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='response' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='":' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' "' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='Hello' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='!' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' How' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' can' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' I' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' help' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' you' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content=' today' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='?"' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='}' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='\n' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='\n' id='run-93221631-62d1-455e-a68c-5a02d64e8034'
content='\n' id='run-93221631-62d1-4

### This module demonstrates the use of Langchain to generate a response based on a given context and question.
It uses the Ollama model to process the input and return a structured output in JSON format.


``` from langchain_experimental.llms.ollama_functions import OllamaFunctions ```
##### this method can also be used but it is DEPRECATED

In [116]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
# Define the Person schema
class Person(BaseModel):
    name: str = Field(description="Name of the person", required=True)
    height: float = Field(description="The person's height in floating type",
                          required=True)
    hair_color: str = Field(description="The person's hair color", required=False)
# Provide the context and question
context = """
Alex is 5 feet tall.
Claudia is 1 foot taller than Alex and jumps higher than he does.
Claudia is brunette and Alex is blonde.
"""
question = "Who is taller?"
# Define the LLaMA template with explicit instructions to include all required fields
# this template can be used for all the LLama family models and Hermes family models
# You can find the template for an LLM in the realease note of the model in the 
# OFFICIAL webpage or ollama hub
llama3_template = '''<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a smart assistant. Based on the following context and question,
provide a detailed JSON response 
including all required fields according to the schema below.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
CONTEXT: {context}
QUESTION: {question}
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>'''
Prompt = PromptTemplate.from_template(llama3_template)
# Initialize the ChatOllama model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    format='json',
)
# Define structured output using the Person schema
structured_llm = llm.with_structured_output(Person)
# Create the chain
chain = Prompt | structured_llm
# Invoke the chain with the question and context
response = chain.invoke({
    "question": question,
    "context": context
})
print(response)

name='Claudia' height=6.0 hair_color='brunette'


### Using Bind Tools method
#### Note

Hermes may not always work correctly with some methods, and other models might also experience similar issues.

In [69]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(
        model="mistral:instruct",
        format='json',
    )
model = model.bind_tools(
    tools=[
        {
            "name":"get_current_weather",
            "description":"Get the current weather in a give location",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":'string',
                        "description":"The city and state, e.g. San Francisco, CA",
                    },
                    "unit":{
                        'type':'string',
                        "enum":['celsius','fahrenheit'],
                    },
                },
                "required":["location","unit"],
            }
        }
    ],
)

response = model.invoke(" what is the weather in Singapore?")
print(response.tool_calls[0]['name'],"with arguments",response.tool_calls[0]['args'])

get_current_weather with arguments {'location': 'Singapore', 'unit': 'celsius'}


### Recommended Method

This is the best and easiest method for function calling that I recommend.


In [118]:
from langchain_core.tools import tool
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def subtract(a: int, b: int) -> int:
    """Subtracts a and b."""
    return a - b

@tool
def devide(a: float, b: float) -> float:
    """Devides a and b."""
    return a / b

def use_Internet(a:str)->str:
    """ Use internet if there is a shortage of information or
     anything related to upto date information"""
    return a

def date(a:str)->str:
    """ Tells todays date and time """
    return a

tools = [add, subtract, multiply, devide, use_Internet, date]

from langchain_ollama.chat_models import ChatOllama
model =ChatOllama(model="hermes3")
llm_with_tools = model.bind_tools(tools)

query = "What is 3 * 12"

print(llm_with_tools.invoke(query).tool_calls)
query = "What is todays special news in india?"

print(llm_with_tools.invoke(query).tool_calls)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '70a16d81-9161-47c2-8b4d-c570e748cee6', 'type': 'tool_call'}]
[{'name': 'use_Internet', 'args': {'a': 'special news in India'}, 'id': '069946cd-8b16-454d-ae4f-3569d7ad25e1', 'type': 'tool_call'}]


<div style="text-align: center;">
    <p style="font-size: 1em; margin: 0;">Written by Ramachandra Udupa | <a href="https://www.linkedin.com/in/ramachandra-udupa/" style="color: #4CAF50; text-decoration: none;">Connect on LinkedIn</a></p>
</div>